# Setup a TiTiler and TiPg services

This notebook contains the code to set up a TiTiler and TiPg services.

## Setup

### Install TiTiler from PyPI and run it locally

Make sure you have pip up to date
```bash
python -m pip install -U pip
```

Install TiTiler from PyPI
```bash
python -m pip  install titiler.{package}
```
e.g.,
```bash
python -m pip  install titiler.core
python -m pip  install titiler.extensions
python -m pip  install titiler.mosaic
python -m pip  install titiler.application # also installs core, extensions and mosaic
```

Install uvicorn to run the FastAPI application locally
```bash
python -m pip install uvicorn
```

Launch application locally
```bash
uvicorn titiler.application.main:app
```

Alternativelly, you can create a [create a FastAPI router](https://developmentseed.org/titiler/advanced/tiler_factories/) (`fastapi.APIRouter`) with a minimal set of endpoints.

You can run our custom app from the src folder with:
```bash
python titiler_app.py
```

### Install TiPg from PyPI and run it locally


### Library import

In [1]:
import json

import httpx
from folium import Map, TileLayer

## Explore the data
### Explore the COGs with TiTiler

In [2]:
raster_data_path = "../data/raw/raster_data/lci_cog_20231016_01.tif"

titiler_endpoint = "http://127.0.0.1:8080"  # Make sure to run the Titiler server locally

**Get COG Info**

In [3]:
# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
try:
    r = httpx.get(
        f"{titiler_endpoint}/info",
        params={
            "url": raster_data_path,
        },
    ).json()

    bounds = r["bounds"]
    print(json.dumps(r, indent=4))

except httpx.HTTPError as e:
    print(f"HTTP error occurred: {e}")

except Exception as e:
    print(f"An error occurred: {e}")

{
    "bounds": [
        -180.0,
        -61.60639637138605,
        179.99999999999935,
        85.05112877980659
    ],
    "minzoom": 0,
    "maxzoom": 5,
    "band_metadata": [
        [
            "b1",
            {}
        ],
        [
            "b2",
            {}
        ],
        [
            "b3",
            {}
        ],
        [
            "b4",
            {}
        ]
    ],
    "band_descriptions": [
        [
            "b1",
            "index_b1"
        ],
        [
            "b2",
            "index_b2"
        ],
        [
            "b3",
            "index_b3"
        ],
        [
            "b4",
            "index_b4"
        ]
    ],
    "dtype": "float32",
    "nodata_type": "Nodata",
    "colorinterp": [
        "gray",
        "undefined",
        "undefined",
        "undefined"
    ],
    "driver": "GTiff",
    "count": 4,
    "width": 8192,
    "height": 5888,
    "overviews": [
        2,
        4,
        8,
        16
    ],
    "nod

**Display Tiles**

Using the `expression` parameter you can apply a custom formula to the data. The formula is applied to each pixel in each band and the result is used to create the output image. The formula is written in [numexpr](https://numexpr.readthedocs.io/en/latest/index.html) syntax.

In [4]:
r = httpx.get(
    f"{titiler_endpoint}/tilejson.json",
    params={
        "url": raster_data_path,
        "bidx": [1, 2, 3, 4],
        "rescale": "0,100",
        "expression": "(b1+b2+b3+b4)/4",
        "colormap_name": "schwarzwald",
    },
).json()

In [6]:
m = Map(location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom_start=3)

TileLayer(tiles=r["tiles"][0], opacity=1, attr="LCI").add_to(m)

m.show_in_browser()

Your map should have been opened in your browser automatically.
Press ctrl+c to return.
